In [1]:
# =============================================================================
# IMPORTAÇÕES E CONFIGURAÇÕES INICIAIS
# =============================================================================

import psycopg2
import os
import requests
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
from psycopg2 import OperationalError, InterfaceError, DatabaseError
import time
from contextlib import contextmanager
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("📚 Bibliotecas importadas com sucesso!")

📚 Bibliotecas importadas com sucesso!


In [2]:
# =============================================================================
# CONFIGURAÇÃO DE VARIÁVEIS DE AMBIENTE
# =============================================================================

# Carregar configurações do ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("API_KEY")
WEATHER_CITIES = os.getenv("WEATHER_CITY", "curitiba,sao paulo,rio de janeiro,brasilia,recife").split(",")
WEATHER_UNITS = os.getenv("WEATHER_UNITS", "metric")

# Parâmetros de conexão do banco
DB_PARAMS = {
    "host": os.getenv("DB_HOST", "localhost"),
    "port": os.getenv("DB_PORT", "5433"),
    "database": os.getenv("DB_NAME", "weather"),
    "user": os.getenv("DB_USER", "postgres"),
    "password": os.getenv("DB_PASSWORD", "root")
}

print(f"🔑 API Key configurada: {'✅' if API_KEY else '❌'}")
print(f"🏙️ Cidades configuradas: {[city.strip() for city in WEATHER_CITIES]}")
print(f"🌡️ Unidades: {WEATHER_UNITS}")

🔑 API Key configurada: ❌
🏙️ Cidades configuradas: ['curitiba', 'sao paulo', 'rio de janeiro', 'brasilia', 'recife']
🌡️ Unidades: metric


In [3]:
# =============================================================================
# CLASSE PARA GERENCIAR CONEXÃO COM O BANCO
# =============================================================================

class ImprovedDatabaseManager:
    def __init__(self, db_params):
        self.db_params = db_params
        self.conn = None
        self._max_retries = 3
        self._retry_delay = 2
    
    @contextmanager
    def get_connection(self):
        """Context manager para conexões seguras"""
        conn = None
        try:
            conn = psycopg2.connect(**self.db_params)
            conn.autocommit = False
            yield conn
        except Exception as e:
            if conn:
                try:
                    conn.rollback()
                except Exception:
                    pass
            raise e
        finally:
            if conn and not conn.closed:
                try:
                    conn.close()
                except Exception as e:
                    logger.warning(f"⚠️ Erro ao fechar conexão: {e}")
    
    def test_connection(self):
        """Testa a conexão com o banco"""
        try:
            with self.get_connection() as conn:
                with conn.cursor() as cursor:
                    cursor.execute("SELECT 1")
                    cursor.fetchone()
                    conn.commit()
            logger.info("✅ Conexão com PostgreSQL testada com sucesso!")
            return True
        except Exception as e:
            logger.error(f"❌ Erro ao testar conexão: {e}")
            return False
    
    def execute_query(self, query, params=None, fetch=False, commit=True):
        """Executa uma query com tratamento robusto de erros"""
        for attempt in range(self._max_retries):
            try:
                with self.get_connection() as conn:
                    with conn.cursor() as cursor:
                        cursor.execute(query, params)
                        
                        result = None
                        if fetch:
                            result = cursor.fetchall()
                        
                        if commit:
                            conn.commit()
                        
                        return result if fetch else True
                        
            except (OperationalError, InterfaceError, DatabaseError) as e:
                logger.warning(f"⚠️ Erro de banco na tentativa {attempt + 1}: {e}")
                if attempt < self._max_retries - 1:
                    time.sleep(self._retry_delay)
                    continue
                else:
                    logger.error(f"❌ Falha após {self._max_retries} tentativas")
                    return None
                    
            except Exception as e:
                logger.error(f"❌ Erro inesperado: {e}")
                return None
        
        return None
    
    def execute_batch(self, queries_and_params):
        """Executa múltiplas queries em uma única transação"""
        try:
            with self.get_connection() as conn:
                with conn.cursor() as cursor:
                    results = []
                    for query_info in queries_and_params:
                        if isinstance(query_info, tuple) and len(query_info) >= 2:
                            query, params = query_info[0], query_info[1]
                            fetch = query_info[2] if len(query_info) > 2 else False
                        else:
                            query = query_info
                            params = None
                            fetch = False
                        
                        cursor.execute(query, params)
                        
                        if fetch:
                            results.append(cursor.fetchall())
                        else:
                            results.append(True)
                    
                    conn.commit()
                    return results
                    
        except Exception as e:
            logger.error(f"❌ Erro na execução em lote: {e}")
            return None

    def get_table_columns(self, table_name):
        """Obtém as colunas de uma tabela"""
        query = """
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = %s 
        ORDER BY ordinal_position
        """
        result = self.execute_query(query, (table_name,), fetch=True)
        return result if result else []

In [4]:
# Inicializar gerenciador melhorado do banco
db = ImprovedDatabaseManager(DB_PARAMS)
if db.test_connection():
    print("🎯 Gerenciador de banco inicializado e testado com sucesso!")
else:
    raise Exception("❌ Não foi possível inicializar o gerenciador de banco")

2025-05-24 16:08:41,602 - INFO - ✅ Conexão com PostgreSQL testada com sucesso!


🎯 Gerenciador de banco inicializado e testado com sucesso!


In [5]:
# =============================================================================
# FUNÇÃO PARA BUSCAR DADOS DA API
# =============================================================================

def fetch_weather_data(city, api_key, units="metric"):
    """Busca dados climáticos atuais de uma cidade"""
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    
    params = {
        'q': city.strip(),
        'appid': api_key,
        'units': units,
        'lang': 'pt_br'
    }
    
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # Extrair dados relevantes
        weather_info = {
            'city': data['name'],
            'country': data['sys']['country'],
            'temperature': round(data['main']['temp'], 1),
            'humidity': data['main']['humidity'],
            'pressure': data['main'].get('pressure'),  # Usar .get() para evitar erro se não existir
            'description': data['weather'][0]['description'],
            'timestamp': datetime.now()
        }
        
        return weather_info
        
    except requests.exceptions.RequestException as e:
        logger.error(f"❌ Erro na requisição para {city}: {e}")
        return None
    except KeyError as e:
        logger.error(f"❌ Dados incompletos na resposta da API para {city}: {e}")
        return None
    except Exception as e:
        logger.error(f"❌ Erro inesperado ao buscar dados para {city}: {e}")
        return None

In [6]:
# Testar a função com uma cidade
print("🧪 Testando conexão com a API...")
if API_KEY and WEATHER_CITIES:
    test_data = fetch_weather_data(WEATHER_CITIES[0], API_KEY, WEATHER_UNITS)
    if test_data:
        print(f"✅ Teste bem-sucedido! Dados de {test_data['city']}: {test_data['temperature']}°C, {test_data['humidity']}%")
    else:
        print("❌ Falha no teste da API")
else:
    print("⚠️ API Key ou cidades não configuradas")

🧪 Testando conexão com a API...
✅ Teste bem-sucedido! Dados de Curitiba: 18.1°C, 73%


In [7]:
# =============================================================================
# VERIFICAÇÃO E AJUSTE DA ESTRUTURA DA TABELA
# =============================================================================

print("🔍 Verificando estrutura da tabela existente...")

# Verificar colunas existentes
existing_columns = db.get_table_columns('raw_data')
if existing_columns:
    print("📋 Colunas existentes na tabela raw_data:")
    for col_name, col_type in existing_columns:
        print(f"   - {col_name}: {col_type}")
    
    # Definir colunas necessárias com seus tipos
    required_columns = {
        'description': 'TEXT',
        'country': 'TEXT', 
        'pressure': 'REAL'
    }
    
    # Verificar colunas existentes
    column_names = [col[0] for col in existing_columns]
    
    # Adicionar colunas que estão faltando
    for col_name, col_type in required_columns.items():
        if col_name not in column_names:
            print(f"⚠️ Coluna '{col_name}' não encontrada. Adicionando...")
            add_column_query = f"ALTER TABLE raw_data ADD COLUMN {col_name} {col_type};"
            if db.execute_query(add_column_query):
                print(f"✅ Coluna '{col_name}' adicionada com sucesso!")
            else:
                print(f"❌ Erro ao adicionar coluna '{col_name}'")
        else:
            print(f"✅ Coluna '{col_name}' já existe!")
else:
    print("❌ Não foi possível obter informações da tabela raw_data")

🔍 Verificando estrutura da tabela existente...
📋 Colunas existentes na tabela raw_data:
   - id: integer
   - city: character varying
   - temperature: real
   - humidity: real
   - timestamp: timestamp without time zone
   - pressure: real
   - description: text
   - country: text
✅ Coluna 'description' já existe!
✅ Coluna 'country' já existe!
✅ Coluna 'pressure' já existe!


In [8]:
# =============================================================================
# CRIAÇÃO DE ÍNDICES ADICIONAIS (SE NECESSÁRIO)
# =============================================================================

print("🏗️ Verificando e criando índices necessários...")

# Índices que queremos garantir que existam
additional_indexes = [
    ("CREATE INDEX IF NOT EXISTS idx_raw_data_city ON raw_data(city);", "idx_raw_data_city"),
    ("CREATE INDEX IF NOT EXISTS idx_raw_data_timestamp ON raw_data(timestamp);", "idx_raw_data_timestamp"),
    ("CREATE INDEX IF NOT EXISTS idx_raw_data_city_timestamp ON raw_data(city, timestamp);", "idx_raw_data_city_timestamp"),
]

for index_query, index_name in additional_indexes:
    if db.execute_query(index_query):
        print(f"✅ Índice {index_name} criado/verificado")
    else:
        print(f"❌ Erro ao criar índice {index_name}")

🏗️ Verificando e criando índices necessários...
✅ Índice idx_raw_data_city criado/verificado
✅ Índice idx_raw_data_timestamp criado/verificado
✅ Índice idx_raw_data_city_timestamp criado/verificado


In [9]:
# =============================================================================
# INSERÇÃO DE DADOS REAIS DA API
# =============================================================================

print("🌐 Buscando dados climáticos reais das cidades configuradas...")

# Query de inserção - usando apenas as colunas que existem na tabela
# Vamos verificar quais colunas existem antes de fazer o INSERT
print("🔍 Verificando colunas para INSERT...")

# Obter colunas atualizadas após as alterações
updated_columns = db.get_table_columns('raw_data')
available_columns = [col[0] for col in updated_columns] if updated_columns else []

print(f"📋 Colunas disponíveis: {available_columns}")

# Construir INSERT baseado nas colunas disponíveis
base_columns = ['city', 'temperature', 'humidity', 'timestamp']
optional_columns = ['description', 'country', 'pressure']

# Colunas que realmente vamos usar
insert_columns = []
insert_values = []

for col in base_columns:
    if col in available_columns:
        insert_columns.append(col)
        insert_values.append('%s')

for col in optional_columns:
    if col in available_columns:
        insert_columns.append(col)
        insert_values.append('%s')

# Construir a query dinamicamente
insert_query = f"""
INSERT INTO raw_data ({', '.join(insert_columns)})
VALUES ({', '.join(insert_values)});
"""

print(f"🔧 Query de inserção gerada: {insert_query}")

🌐 Buscando dados climáticos reais das cidades configuradas...
🔍 Verificando colunas para INSERT...
📋 Colunas disponíveis: ['id', 'city', 'temperature', 'humidity', 'timestamp', 'pressure', 'description', 'country']
🔧 Query de inserção gerada: 
INSERT INTO raw_data (city, temperature, humidity, timestamp, description, country, pressure)
VALUES (%s, %s, %s, %s, %s, %s, %s);



In [10]:
successful_inserts = 0
failed_inserts = 0

if API_KEY and WEATHER_CITIES:
    for city in WEATHER_CITIES:
        print(f"🏙️ Processando {city.strip()}...")
        
        # Buscar dados da API
        weather_data = fetch_weather_data(city, API_KEY, WEATHER_UNITS)
        
        if weather_data:
            # Construir tupla de valores baseada nas colunas disponíveis
            values = []
            
            for col in insert_columns:
                if col == 'city':
                    values.append(weather_data['city'])
                elif col == 'temperature':
                    values.append(weather_data['temperature'])
                elif col == 'humidity':
                    values.append(weather_data['humidity'])
                elif col == 'timestamp':
                    values.append(weather_data['timestamp'])
                elif col == 'description':
                    values.append(weather_data.get('description'))
                elif col == 'country':
                    values.append(weather_data.get('country'))
                elif col == 'pressure':
                    values.append(weather_data.get('pressure'))
            
            # Inserir no banco
            success = db.execute_query(insert_query, tuple(values))
            
            if success:
                print(f"   ✅ {weather_data['city']}: {weather_data['temperature']}°C, {weather_data['humidity']}% umidade")
                successful_inserts += 1
            else:
                print(f"   ❌ Erro ao inserir dados de {city}")
                failed_inserts += 1
        else:
            print(f"   ❌ Não foi possível obter dados de {city}")
            failed_inserts += 1
        
        # Pequena pausa para respeitar rate limits da API
        time.sleep(1)
else:
    print("⚠️ API Key ou cidades não configuradas. Pulando inserção de dados.")

print(f"\n📊 Resumo da inserção:")
print(f"   ✅ Sucessos: {successful_inserts}")
print(f"   ❌ Falhas: {failed_inserts}")
total_attempts = successful_inserts + failed_inserts
if total_attempts > 0:
    print(f"   📈 Taxa de sucesso: {(successful_inserts/total_attempts*100):.1f}%")

🏙️ Processando curitiba...
   ✅ Curitiba: 18.1°C, 73% umidade
🏙️ Processando sao paulo...
   ✅ São Paulo: 20.2°C, 80% umidade
🏙️ Processando rio de janeiro...
   ✅ Rio de Janeiro: 23.5°C, 59% umidade
🏙️ Processando brasilia...
   ✅ Brasília: 25.5°C, 49% umidade
🏙️ Processando recife...
   ✅ Recife: 28.0°C, 78% umidade

📊 Resumo da inserção:
   ✅ Sucessos: 5
   ❌ Falhas: 0
   📈 Taxa de sucesso: 100.0%


In [11]:
# =============================================================================
# CRIAÇÃO DE TABELA MODEL_PREDICTIONS (SE NÃO EXISTIR)
# =============================================================================

print("🤖 Verificando tabela model_predictions...")

create_predictions_table = """
CREATE TABLE IF NOT EXISTS model_predictions (
    id SERIAL PRIMARY KEY,
    city VARCHAR(100),
    humidity REAL NOT NULL,
    predicted_temperature REAL NOT NULL,
    actual_temperature REAL,
    prediction_error REAL,
    model_version VARCHAR(50),
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

if db.execute_query(create_predictions_table):
    print("✅ Tabela model_predictions criada/verificada")
    
    # Criar índices para model_predictions
    predictions_indexes = [
        "CREATE INDEX IF NOT EXISTS idx_predictions_city ON model_predictions(city);",
        "CREATE INDEX IF NOT EXISTS idx_predictions_timestamp ON model_predictions(timestamp);",
        "CREATE INDEX IF NOT EXISTS idx_predictions_model_version ON model_predictions(model_version);",
    ]
    
    for idx_query in predictions_indexes:
        db.execute_query(idx_query)
else:
    print("❌ Erro ao criar tabela model_predictions")

🤖 Verificando tabela model_predictions...
✅ Tabela model_predictions criada/verificada


In [12]:
# =============================================================================
# CRIAÇÃO DE VIEWS E RELATÓRIOS
# =============================================================================

print("🔍 Criando views para análises...")

# Criar view para análises (com tratamento para pressure que pode ser NULL)
create_view_query = """
CREATE OR REPLACE VIEW weather_summary AS
SELECT 
    city,
    DATE(timestamp) as date,
    COUNT(*) as measurements,
    ROUND(AVG(temperature)::numeric, 1) as avg_temperature,
    ROUND(MIN(temperature)::numeric, 1) as min_temperature,
    ROUND(MAX(temperature)::numeric, 1) as max_temperature,
    ROUND(AVG(humidity)::numeric, 1) as avg_humidity,
    ROUND(AVG(COALESCE(pressure, 0))::numeric, 1) as avg_pressure
FROM raw_data
GROUP BY city, DATE(timestamp)
ORDER BY city, date DESC;
"""

if db.execute_query(create_view_query):
    print("✅ View 'weather_summary' criada para análises")
else:
    print("❌ Erro ao criar view 'weather_summary'")

🔍 Criando views para análises...
✅ View 'weather_summary' criada para análises


In [13]:
# =============================================================================
# GERAÇÃO DE RELATÓRIOS
# =============================================================================

print("📊 Gerando relatório dos dados inseridos...")

try:
    # Contar registros totais
    total_result = db.execute_query("SELECT COUNT(*) FROM raw_data", fetch=True)
    total_records = total_result[0][0] if total_result and total_result[0] else 0
    print(f"\n📈 Total de registros na tabela raw_data: {total_records}")

    if total_records > 0:
        # Estatísticas por cidade
        city_stats_query = """
        SELECT 
            city,
            COUNT(*) as registros,
            ROUND(AVG(temperature)::numeric, 1) as temp_media,
            ROUND(AVG(humidity)::numeric, 1) as umidade_media,
            MIN(timestamp) as primeiro_registro,
            MAX(timestamp) as ultimo_registro
        FROM raw_data 
        GROUP BY city 
        ORDER BY registros DESC
        """
        
        city_stats = db.execute_query(city_stats_query, fetch=True)
        
        if city_stats:
            print("\n🏙️ Estatísticas por cidade:")
            print("-" * 80)
            for stat in city_stats:
                city, count, avg_temp, avg_hum, first, last = stat
                first_str = first.strftime('%d/%m %H:%M') if first else "N/A"
                last_str = last.strftime('%d/%m %H:%M') if last else "N/A"
                print(f"{city:15} | {count:3d} reg | {avg_temp:5.1f}°C | {avg_hum:5.1f}% | {first_str} - {last_str}")

        # Últimos dados inseridos
        recent_query = """
        SELECT city, temperature, humidity, description, timestamp 
        FROM raw_data 
        ORDER BY timestamp DESC 
        LIMIT 5
        """
        
        recent_data = db.execute_query(recent_query, fetch=True)
        
        if recent_data:
            print("\n🔍 Últimos 5 registros inseridos:")
            print("-" * 80)
            for record in recent_data:
                city, temp, hum, desc, ts = record
                ts_str = ts.strftime('%d/%m %H:%M:%S') if ts else "N/A"
                desc_str = (desc or "N/A")[:20]  # Tratar None e limitar tamanho
                print(f"{city:15} | {temp:5.1f}°C | {hum:3.0f}% | {desc_str:20} | {ts_str}")

except Exception as e:
    logger.error(f"❌ Erro ao gerar relatório: {e}")

📊 Gerando relatório dos dados inseridos...

📈 Total de registros na tabela raw_data: 27

🏙️ Estatísticas por cidade:
--------------------------------------------------------------------------------
Curitiba        |  11 reg |  18.4°C |  78.4% | 24/05 11:34 - 24/05 16:09
Rio de Janeiro  |   4 reg |  22.3°C |  75.3% | 24/05 11:34 - 24/05 16:09
Brasília        |   4 reg |  24.3°C |  52.0% | 24/05 11:34 - 24/05 16:09
Recife          |   4 reg |  28.8°C |  76.3% | 24/05 11:34 - 24/05 16:09
São Paulo       |   4 reg |  20.2°C |  76.5% | 24/05 11:34 - 24/05 16:09

🔍 Últimos 5 registros inseridos:
--------------------------------------------------------------------------------
Recife          |  28.0°C |  78% | nuvens dispersas     | 24/05 16:09:09
Brasília        |  25.5°C |  49% | nublado              | 24/05 16:09:08
Rio de Janeiro  |  23.5°C |  59% | nublado              | 24/05 16:09:07
São Paulo       |  20.2°C |  80% | nublado              | 24/05 16:09:05
Curitiba        |  18.1°C |  7

In [14]:
# =============================================================================
# VALIDAÇÃO FINAL
# =============================================================================

print("\n🔍 Validação final do banco de dados...")

# Verificar se as tabelas foram criadas corretamente
validation_queries = [
    ("SELECT COUNT(*) FROM information_schema.tables WHERE table_name = 'raw_data'", "Tabela raw_data"),
    ("SELECT COUNT(*) FROM information_schema.tables WHERE table_name = 'model_predictions'", "Tabela model_predictions"),
    ("SELECT COUNT(*) FROM information_schema.views WHERE table_name = 'weather_summary'", "View weather_summary"),
    ("SELECT COUNT(*) FROM raw_data", "Registros em raw_data"),
]

all_validations_passed = True

for query, description in validation_queries:
    try:
        result = db.execute_query(query, fetch=True)
        count = result[0][0] if result and result[0] else 0
        
        if "Registros" in description:
            print(f"✅ {description}: {count}")
        elif count > 0:
            print(f"✅ {description}: Existe")
        else:
            print(f"❌ {description}: Não encontrado")
            all_validations_passed = False
            
    except Exception as e:
        print(f"❌ Erro ao validar {description}: {e}")
        all_validations_passed = False

# Resultado final
print("\n" + "="*60)
if all_validations_passed:
    print("🎉 CONFIGURAÇÃO CONCLUÍDA COM SUCESSO!")
    print("✅ Banco de dados estruturado e funcionando corretamente")
    print("✅ Dados reais inseridos da API OpenWeatherMap")
    print("✅ Views de análise criadas")
    print("✅ Sistema pronto para uso!")
else:
    print("⚠️ CONFIGURAÇÃO PARCIALMENTE CONCLUÍDA")
    print("⚠️ Alguns componentes podem não estar funcionando corretamente")
    print("⚠️ Verifique os logs acima para mais detalhes")

print("="*60)
print("🎯 Configuração do banco de dados finalizada!")


🔍 Validação final do banco de dados...
✅ Tabela raw_data: Existe
✅ Tabela model_predictions: Existe
✅ View weather_summary: Existe
✅ Registros em raw_data: 27

🎉 CONFIGURAÇÃO CONCLUÍDA COM SUCESSO!
✅ Banco de dados estruturado e funcionando corretamente
✅ Dados reais inseridos da API OpenWeatherMap
✅ Views de análise criadas
✅ Sistema pronto para uso!
🎯 Configuração do banco de dados finalizada!
